In [1]:
%pip install -q altair

StatementMeta(, 022b8b8e-6d1a-41ae-a177-1782400164ee, 7, Finished, Available, Finished)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires sentencepiece, which is not installed.
sentence-transformers 2.0.0 requires torchvision, which is not installed.
dash 2.14.0 requires Flask<2.3.0,>=1.0.4, but you have flask 3.0.0 which is incompatible.
dash 2.14.0 requires Werkzeug<2.3.0, but you have werkzeug 3.0.1 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
try :
    spark.getActiveSession()
    from pyspark.sql  import functions as f
    base_path = "Tables/"
    print('session active')
except:
    !pip install -q "sqlframe[duckdb]"
    from sqlframe.duckdb.session import DuckDBSession
    from sqlframe.duckdb import functions as f
    spark = DuckDBSession()
    base_path = "/lakehouse/default/Tables/"
    print('session starting')

StatementMeta(, 022b8b8e-6d1a-41ae-a177-1782400164ee, 9, Finished, Available, Finished)

session active


In [3]:
df = spark.read.load(base_path+"ny/taxi",format ="delta")
df.count()

StatementMeta(, 022b8b8e-6d1a-41ae-a177-1782400164ee, 10, Finished, Available, Finished)

17804016

In [4]:
%%time
result = df.groupBy("date").agg( f.sum("fare_amount").alias("totalfares"),f.avg("fare_amount").alias("avgfares") )
data = result.toPandas()

StatementMeta(, 022b8b8e-6d1a-41ae-a177-1782400164ee, 11, Finished, Available, Finished)

CPU times: user 75.1 ms, sys: 4.02 ms, total: 79.1 ms
Wall time: 4.63 s


In [5]:
import altair as alt
brush = alt.selection_interval()
details = alt.Chart(data).mark_bar().encode(alt.X('date:T'),  alt.Y('totalfares:Q'), tooltip=[alt.Tooltip('date:T',format='%Y-%m-%d %H'),'totalfares:Q']
).properties( width=1400,  height=400 ).add_params( brush)

summary = alt.Chart(data).mark_square().encode( alt.X('date:T'), alt.Y('avgfares:Q'), tooltip=['avgfares:Q'] ).properties(  width=1400,  height=400).transform_filter( brush)
details & summary

StatementMeta(, 022b8b8e-6d1a-41ae-a177-1782400164ee, 12, Finished, Available, Finished)

alt.VConcatChart(...)